In [2]:
# Import all Required Libraries
import pandas as pd
import re

### Loading the Data

In [3]:
# Read data from a csv file
mf = pd.read_csv('MF_Data.csv')

In [4]:
mf.head()

,AMC Name,AMC Launch Date,Rank,Mutual Fund Name,AUM,Type,Scheme Launch Date,Exit Load,Expense Ratio,Investment Type,...,Minimum Investment,Minimum SIP Amount,Total Return,1m_return,1y_return,2y_return,3y_return,4y_return,5y_return,10y_return
0,ESCORTS,1996-04-15T00:00:00.822Z,34.0,Quant Small Cap Fund Direct Plan Growth,5565.264021,Equity,07-Jan-2013,Exit load of 1% if redeemed within 1 year,0.62,Small Cap,...,5000,1000.0,433.995802,7.212296,42.029294,40.335026,283.355608,324.014833,251.597822,414.830298
1,RELIANCE,1995-06-30T00:00:00.684Z,6.0,Nippon India Small Cap Fund Direct Growth,31945.148000,Equity,31-Dec-2012,Exit load of 1% if redeemed within one month.,0.75,Small Cap,...,5000,100.0,1043.195148,6.097628,37.840897,51.430616,223.395720,225.251920,187.826655,1273.914846
2,ESCORTS,1996-04-15T00:00:00.459Z,34.0,Quant Infrastructure Fund Direct Growth,924.574701,Equity,07-Jan-2013,Exit load of 0.50% if redeemed within 3 months.,0.64,Sectoral / Thematic,...,5000,1000.0,405.316648,6.683960,21.283722,46.701603,240.254070,204.645633,211.893923,616.427861
3,ICICI_PRUDENTIAL,1993-10-12T00:00:00.628Z,3.0,ICICI Prudential Commodities Fund Direct Growth,994.134033,Equity,15-Oct-2019,Exit load of 1% if redeemed within 3 months.,1.06,Sectoral / Thematic,...,5000,100.0,214.800000,4.968323,30.622407,25.518341,219.269777,NaN,NaN,NaN
4,HSBC,2002-05-27T00:00:00.707Z,21.0,HSBC Small Cap Fund Direct Growth,10129.408282,Equity,12-May-2014,"For units in excess of 10% of the investment,1...",0.73,Small Cap,...,5000,1000.0,521.505000,5.097216,32.283798,46.487740,209.226467,168.903109,130.619235,NaN


In [5]:
mf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169 entries, 0 to 1168
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AMC Name            1167 non-null   object 
 1   AMC Launch Date     1166 non-null   object 
 2   Rank                1141 non-null   float64
 3   Mutual Fund Name    1169 non-null   object 
 4   AUM                 1158 non-null   float64
 5   Type                1169 non-null   object 
 6   Scheme Launch Date  1167 non-null   object 
 7   Exit Load           1169 non-null   object 
 8   Expense Ratio       1144 non-null   float64
 9   Investment Type     1169 non-null   object 
 10  NAV                 1169 non-null   float64
 11  Minimum Investment  1169 non-null   int64  
 12  Minimum SIP Amount  1160 non-null   float64
 13  Total Return        1139 non-null   float64
 14  1m_return           1133 non-null   float64
 15  1y_return           1027 non-null   float64
 16  2y_ret

In [6]:
# Check Duplicate Values
mf[mf.duplicated()].shape

(56, 21)

### Remove Duplicates

In [7]:
mf.drop_duplicates(inplace=True)

In [8]:
mf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1113 entries, 0 to 1168
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AMC Name            1111 non-null   object 
 1   AMC Launch Date     1110 non-null   object 
 2   Rank                1086 non-null   float64
 3   Mutual Fund Name    1113 non-null   object 
 4   AUM                 1102 non-null   float64
 5   Type                1113 non-null   object 
 6   Scheme Launch Date  1111 non-null   object 
 7   Exit Load           1113 non-null   object 
 8   Expense Ratio       1092 non-null   float64
 9   Investment Type     1113 non-null   object 
 10  NAV                 1113 non-null   float64
 11  Minimum Investment  1113 non-null   int64  
 12  Minimum SIP Amount  1104 non-null   float64
 13  Total Return        1083 non-null   float64
 14  1m_return           1077 non-null   float64
 15  1y_return           1003 non-null   float64
 16  2y_return  

### Extract Exit load percentage and duration from Text

In [9]:
mf['Exit Load'].unique()

array(['Exit load of 1% if redeemed within 1 year',
       'Exit load of 1% if redeemed within one month.',
       'Exit load of 0.50% if redeemed within 3 months.',
       'Exit load of 1% if redeemed within 3 months.',
       'For units in excess of 10% of the investment,1% will be charged for redemption within 1 year.',
       'Exit load of 1% if redeemed within 15 days',
       'For units more than 12% of the investments, an exit load of 1% if redeemed within 12 months.',
       'Exit load of 1% if redeemed upto 1 year',
       'Exit load of 1%, if redeemed within 30 days.',
       'Exit load of 1% if redeemed within 1 year.',
       'For units in excess of 10% of the investment,1% will be charged for redemption within 365 days',
       'Exit load of 1%, if redeemed within 90 days.',
       'Exit load of 1% if redeemed within 1 year for units beyond 10% of the investment.',
       'If unit beyond 10%, Exit load of 1% if redeemed within 1 year.',
       'Exit load of 1% if redeemed 

In [10]:
# Get exitload % from text using Regex
def func_exitload(x):
    f = re.findall('(\d+.?\d*)%',x)
    if f: 
        if float(f[-1])>=0.1:
            if float(f[-1]) >= 10:
                return float(1.0)
            else:
                return float(f[-1])
        else:
            return float(max(f))
    else:
        return float(0)

mf['Exit Load Per'] = mf['Exit Load'].apply(func_exitload)

In [11]:
# Get duration from Text
def func_period(x):
    f = re.findall('(one|\d+) (\w+)',x)
    pro=1
    if f:
        if f[-1][1][0].lower() == 'y':
            pro = 365
        if f[-1][1][0].lower() == 'm':
            pro = 30
        if f[-1][1][0].lower() == 'd':
            pro = 1
        if f[0][0] == 'one':
            return 1*pro
        else:
            return int(f[-1][0])*pro
    else:
        return int('0')
mf['Exit Load Days'] = mf['Exit Load'].apply(func_period)

In [12]:
mf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1113 entries, 0 to 1168
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AMC Name            1111 non-null   object 
 1   AMC Launch Date     1110 non-null   object 
 2   Rank                1086 non-null   float64
 3   Mutual Fund Name    1113 non-null   object 
 4   AUM                 1102 non-null   float64
 5   Type                1113 non-null   object 
 6   Scheme Launch Date  1111 non-null   object 
 7   Exit Load           1113 non-null   object 
 8   Expense Ratio       1092 non-null   float64
 9   Investment Type     1113 non-null   object 
 10  NAV                 1113 non-null   float64
 11  Minimum Investment  1113 non-null   int64  
 12  Minimum SIP Amount  1104 non-null   float64
 13  Total Return        1083 non-null   float64
 14  1m_return           1077 non-null   float64
 15  1y_return           1003 non-null   float64
 16  2y_return  

### Dealing Missing values

In [13]:
mf.drop(mf[(mf['Expense Ratio'].isna()) & (mf['Total Return'].isna())].index,axis=0,inplace=True)

In [14]:
mf[(mf['Expense Ratio'].isna())].shape

(14, 23)

In [15]:
mf[(mf['Total Return'].isna())].shape

(23, 23)

In [ ]:
# Based on my Domain Knowledge i decided to fill it with mean value based on type of mutual fund

In [16]:
mf['Expense Ratio'] = mf.groupby('Type')['Expense Ratio'].transform(lambda x: x.fillna(x.mean()))

In [17]:
mf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1106 entries, 0 to 1167
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AMC Name            1104 non-null   object 
 1   AMC Launch Date     1103 non-null   object 
 2   Rank                1079 non-null   float64
 3   Mutual Fund Name    1106 non-null   object 
 4   AUM                 1099 non-null   float64
 5   Type                1106 non-null   object 
 6   Scheme Launch Date  1104 non-null   object 
 7   Exit Load           1106 non-null   object 
 8   Expense Ratio       1106 non-null   float64
 9   Investment Type     1106 non-null   object 
 10  NAV                 1106 non-null   float64
 11  Minimum Investment  1106 non-null   int64  
 12  Minimum SIP Amount  1098 non-null   float64
 13  Total Return        1083 non-null   float64
 14  1m_return           1077 non-null   float64
 15  1y_return           1003 non-null   float64
 16  2y_return  

In [ ]:
# fill all NAN based on column

In [18]:
mf.fillna({'Total Return':0,'1m_return':0,'1y_return':0,'2y_return':0,'3y_return':0,
           '4y_return':0,'5y_return':0,'10y_return':0,'Minimum SIP Amount':1,'AUM':0},inplace=True)

In [ ]:
# fill AMC Name based on fund name

In [19]:
mf.loc[mf['AMC Name'].isna(),['AMC Name']] = 'BNP Paribas'

In [20]:
mf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1106 entries, 0 to 1167
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AMC Name            1106 non-null   object 
 1   AMC Launch Date     1103 non-null   object 
 2   Rank                1079 non-null   float64
 3   Mutual Fund Name    1106 non-null   object 
 4   AUM                 1106 non-null   float64
 5   Type                1106 non-null   object 
 6   Scheme Launch Date  1104 non-null   object 
 7   Exit Load           1106 non-null   object 
 8   Expense Ratio       1106 non-null   float64
 9   Investment Type     1106 non-null   object 
 10  NAV                 1106 non-null   float64
 11  Minimum Investment  1106 non-null   int64  
 12  Minimum SIP Amount  1106 non-null   float64
 13  Total Return        1106 non-null   float64
 14  1m_return           1106 non-null   float64
 15  1y_return           1106 non-null   float64
 16  2y_return  

In [21]:
mf[(mf['AMC Launch Date'].isna()) | (mf['Scheme Launch Date'].isna())]

,AMC Name,AMC Launch Date,Rank,Mutual Fund Name,AUM,Type,Scheme Launch Date,Exit Load,Expense Ratio,Investment Type,...,Total Return,1m_return,1y_return,2y_return,3y_return,4y_return,5y_return,10y_return,Exit Load Per,Exit Load Days
335,BNP Paribas,NaN,NaN,BNP Paribas Multi Cap Fund Direct Growth,561.508000,Equity,02-Jan-2013,"Exit load of 1%, if redeemed within 3 months",1.01,Multi Cap,...,302.759117,-4.269030,20.459536,68.449466,70.409096,69.761139,103.621940,0.000000,1.0,90
542,BNP Paribas,NaN,NaN,BNP Paribas Short Term Fund Direct Growth,247.544800,Debt,02-Jan-2013,Nil,0.35,Short Duration,...,105.104671,0.210283,5.315622,13.589351,23.515512,33.465449,42.400021,0.000000,0.0,0
600,FRANKLIN_TEMPLETON,1996-02-19T00:00:00.405Z,11.0,Franklin India Floating Rate Fund Direct Growth,284.863583,Debt,NaN,Nil,0.20,Floater,...,100.530870,0.746624,7.916500,11.559163,17.216791,25.985435,35.740649,92.966536,0.0,0
959,UNION_KBC,2009-12-30T00:00:00.957Z,27.0,Union Money Market Fund Direct Growth,169.021400,Debt,NaN,Nil,0.29,Money Market,...,10.221620,0.568667,6.679797,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0
1040,360 ONE,NaN,NaN,360 ONE FlexiCap Fund Direct Growth,220.166161,Equity,30-Jun-2023,"Exit load of 1%, if redeemed within 365 days.",0.90,Flexi Cap,...,1.870000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,365


In [ ]:
# Fill missing dates manually

In [22]:
mf.loc[(mf['AMC Launch Date'].isna()),['AMC Launch Date']]=['2007-03-01T00:00:00.405Z','2007-03-01T00:00:00.405Z'
                                                            ,'2008-03-01T00:00:00.405Z']

In [23]:
mf.loc[(mf['Scheme Launch Date'].isna()),['Scheme Launch Date']]=['30-Jan-2013','26-Aug-2021']

In [ ]:
# Remove unwanted columns

In [24]:
mf.drop(['Exit Load','Rank'],axis=1,inplace=True)

In [25]:
mf.drop(mf[mf['Type']=='Commodities'].index, axis=0, inplace=True)

### DataType Conversion

In [26]:
mf['AMC Launch Date'] = mf['AMC Launch Date'].str[:10]

In [27]:
mf['AMC Launch Date'] = pd.to_datetime(mf['AMC Launch Date'])

In [28]:
mf['Scheme Launch Date'] = pd.to_datetime(mf['Scheme Launch Date'])

In [31]:
mf = mf.astype({'Minimum SIP Amount':'int'})

In [32]:
mf = mf.reset_index(drop=True)

In [33]:
mf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104 entries, 0 to 1103
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   AMC Name            1104 non-null   object        
 1   AMC Launch Date     1104 non-null   datetime64[ns]
 2   Mutual Fund Name    1104 non-null   object        
 3   AUM                 1104 non-null   float64       
 4   Type                1104 non-null   object        
 5   Scheme Launch Date  1104 non-null   datetime64[ns]
 6   Expense Ratio       1104 non-null   float64       
 7   Investment Type     1104 non-null   object        
 8   NAV                 1104 non-null   float64       
 9   Minimum Investment  1104 non-null   int64         
 10  Minimum SIP Amount  1104 non-null   int32         
 11  Total Return        1104 non-null   float64       
 12  1m_return           1104 non-null   float64       
 13  1y_return           1104 non-null   float64     

In [34]:
mf.to_csv('MF_Data_Clean.csv',index=False)